# Patient Biopsies

## Goals

1. Which subpopulation of each biopsy is most plastic? 
    - Expectation: We will be able to quantify plasticity of the cancer cells and then compare within each sample to determine which cluster is most plastic.
    - Methods: 
        - **Should we batch correct?**
        - RNA velocity and dimensionality reduction (**what DR do we want to use?**)
        - Plasticity quantification using two metrics (**what do the end states look like? Are there clusters/ what is the multiplicity of eigenvalue 1?**)
    - Possible pitfalls: Not much range in plasticity within a data set. Several regions of the data set have high plasticity. We need to decide what dimensionality reduction to use- PCA space? We proposed working on the UMAP, which will not work well if there are several subpopulations that are discontinuous.
2. Is it consistent across treatments? 
    - We expect that the post-treatment tumors will have subpopulations with higher plasticity. We will want to compare what subpopulations there are and the plasticity of each. 
    - Methods:
        - Identify cancer subtypes to match across samples/treatment.
        - Compare plasticity of each subtype directly.
        - Consider size of each sample and each subpopulation-- **does number of cells confound plasticity calculations?**
    - Pitfalls: we will need to match subpopulations across samples-- for example, if there are A cells before and after treatment, do those change significantly in phenotype or plasticity?
3. How does the presence and proportion of immune cells affect plasticity of cancer cells?

## Read in data and preprocess.
1. Use cell_ids_biopsies.csv to subset data and identify cancer cells versus immune cells.
2. Filter genes, normalize, and log-transform.
3. Visualize using UMAP and/or tSNE.
4. Cluster data using leiden or louvain.
4. Find marker genes for each cluster, as well as expression of SCLC markers.

## RNA velocity calculation and plasticity quantification.
CDX models from the same patient before and after treatment should be combined so they are comparable (i.e. in the same "space"). However, RNA velocity should be calculated independently for each sample.